In [92]:
import folium
import pandas as pd

bundeslaender_geo = r'data/hoch_bundeslaender_deutschland.geojson'
provincies_geo = r'data/provincies_nederland.geojson'
guests_csv = r'data/FLG_Buchungen.csv'
guests = r'data/FLG_Buchungen.xlsx'

guests_df = pd.read_excel(guests, sheetname='Kunden', encoding='iso-8859-1')[['Klantnummer', 'Staat of provincie', 'Land']].dropna()
buchungen_df = pd.read_excel(guests, sheetname='Buchungen', encoding='iso-8859-1')[['Kunden Nr', 'Bnummer', '# Übern.']].dropna()

guests_df.columns = ['clientnumber', 'state', 'country']
buchungen_df.columns = ['clientnumber', 'bookingnumber', 'duration_in_days']

##### CLEANING THE DATA ######
# Make sure the states do not contain any umlauts, because the matching will fail. 
guests_df = guests_df.replace(to_replace='Thüringen', value='Thueringen').replace(to_replace='Baden-Württemberg', 
                                                                                  value='Baden-Wuerttemberg')
# Delete Familie Lubbers and Benning
buchungen_df = buchungen_df[~buchungen_df.clientnumber.isin([0, 1, 82])]
# Also delete stays of 0 days which are Hans' stays (they should not be in there after the previous step)
buchungen_df = buchungen_df[~buchungen_df.duration_in_days.isin([0])]

buchungen_df = buchungen_df.merge(guests_df, left_on='clientnumber', right_on='clientnumber', how='left')
buchungen_counts =  pd.DataFrame(buchungen_df['state'].value_counts()).reset_index()
buchungen_counts.columns = ['state', 'count']



In [102]:
buchungen_mean_duration = buchungen_df.groupby(by='state').mean().reset_index()

In [103]:
buchungen_mean_duration

,state,duration_in_days
0,Baden-Wuerttemberg,7.500000
1,Bayern,7.615385
2,Berlin,5.461538
3,Bern,4.000000
4,Brandenburg,8.150000
5,Hamburg,6.000000
6,Hessen,7.000000
7,Limburg,7.000000
8,Małopolskie,4.000000
9,Mecklenburg-Vorpommern,6.111111


In [106]:
#Let Folium determine the scale
map = folium.Map(location=[51.3, 10], zoom_start=5.5)
# Germany
map.choropleth(geo_path=bundeslaender_geo, data=buchungen_mean_duration,
             columns=['state', 'duration_in_days'],
             key_on='feature.properties.NAME_1',
             fill_color='YlGn', fill_opacity=0.7, line_opacity=0.2,
             legend_name='Durchschnittliche Anzahl Übernachtungen')
# Netherlands
map.choropleth(geo_path=provincies_geo, data=buchungen_mean_duration,
             columns=['state', 'duration_in_days'],
             key_on='feature.properties.name',
             fill_color='YlGn', fill_opacity=0.7, line_opacity=0.2,
             legend_name='Durchschnittliche Anzahl Übernachtungen')
# map
map.save('uebernachtungen.html')

In [95]:
#Let Folium determine the scale
map = folium.Map(location=[51.3, 10], zoom_start=5.5)
# Germany
map.choropleth(geo_path=bundeslaender_geo, data=buchungen_counts,
             columns=['state', 'count'],
             key_on='feature.properties.NAME_1',
             fill_color='YlGn', fill_opacity=0.7, line_opacity=0.2,
             legend_name='Anzahl Buchungen')
# Netherlands
map.choropleth(geo_path=provincies_geo, data=buchungen_counts,
             columns=['state', 'count'],
             key_on='feature.properties.name',
             fill_color='YlGn', fill_opacity=0.7, line_opacity=0.2,
             legend_name='Anzahl Buchungen')
# map
map.save('buchungen.html')